In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from Levenshtein import distance as levenshtein_distance
import pandas as pd
import os
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import re
import numpy as np
import pandas as pd
from pprint import pprint
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy
# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings

warnings.filterwarnings("ignore",category=DeprecationWarning)

# os.chdir("/home/syful-is/softSVM/data/")

df=pd.read_csv("RQ3_relatedness_Filtered_data.csv", encoding = 'latin-1', engine ='python')

df = df.fillna('')

In [144]:
q1_source=[]
q2_source=[]
q1_purpose=[]
q2_purpose=[]
q1_content=[]
q2_content=[]

dict_labels_source = {"Desktop App image":0, "Web browser image":1, "Mobile App image":2, "User-created Image":3}
dict_labels_purpose = {"desired output": 0, "context":1}
dict_labels_content = {"User Interface": 0, "Results":1}

for i in range(len(df['q1_source'])):
    q1_source.append(dict_labels_source[df['q1_source'][i]])
    q2_source.append(dict_labels_source[df['q2_source'][i]])
    q1_purpose.append(dict_labels_purpose[df['q1_purpose'][i]])
    q2_purpose.append(dict_labels_purpose[df['q2_purpose'][i]])
    q1_content.append(dict_labels_content[df['q1_content'][i]])
    q2_content.append(dict_labels_content[df['q2_content'][i]])

In [146]:
import re
import string
import Levenshtein


def convertCamelCase(name):
    s1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    finals = re.sub('([a-z0-9])([A-Z])', r'\1_\2', s1).lower()
    return finals


def text_to_wordlist(text, remove_stopwords=True, stem_words=False, remove_punc=True):
    text = str(text)
    
    text = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', 'URL', text)
    
    text = text.split()
    
    text2 = []
    
    for t in text:
        splited = re.split("[.,' \-!?:#^%*[$\]+|_`\)=<;{\"&>@/~(\\}\\\]+", t)
        text2.extend(splited)
    not_digits=[] 
    
    for i in text2:
        if i.isdigit():
            not_digits.append("cc")
        else:
            not_digits.append(i)
    text4 = []
    for c in not_digits:
        if len(c) > 1:
            if c[1].islower():
                for k in c[1:]:
                    if not k.islower():
                        text4.append(convertCamelCase(c))
                        break;
                else:
                    text4.append(c)
            else:
                text4.append(c)
        else:
            text4.append(c)

    text5 = []
    for t in text4:
        splited = re.split("_", t)
        text5.extend(splited)
    text6=[]
    for x in text5:
        if len(x)>1:
            text6.append(x)

    if remove_stopwords:
        stops = set(stopwords.words("english"))
        text = [w for w in text6 if not w in stops]

    text = " ".join(text)

    text = re.sub("  ", " ", text)
    text = re.sub("   ", " ", text)
    text = re.sub("    ", " ", text)
    text = re.sub("[0-9]+", "CC", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"couldn't", "could not ", text)
    text = re.sub(r"doesn’t", "does not ", text)
    text = re.sub(r"don't", "do not ", text)
    text = re.sub(r"won't", "will not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'", "", text)
    text = re.sub(r"\"", "", text)
    text = re.sub(r"“", "", text)
    text = re.sub(r"”", "", text)

    if remove_punc:
        exclude = set(string.punctuation)
        text = ''.join(ch for ch in text if ch not in exclude)

    if stem_words:
        text = text.split()
        stemmer = SnowballStemmer('english')
        stemmed_words = [stemmer.stem(word) for word in text]
        text = " ".join(stemmed_words)

    text = text.lower()

    return (text)

In [ ]:
q1_image_text = []
q2_image_text = []
q1_process = []
q2_process = []

count = 1
for index, row in df.iterrows():

    q1_Body = text_to_wordlist(str(row['q1_Title'])+ " " +str(row['q1_Body'])+ " " +str(row['q1_AcceptedAnswerBody'])+" " + str(row['Q1_image_text_new']), remove_stopwords=True, stem_words=False, remove_punc=True)
    q1_process.append(q1_Body)
    q1_image_text.append(text_to_wordlist(str(row['Q1_image_text_new']), remove_stopwords=True, stem_words=False, remove_punc=True))

    q2_Body = text_to_wordlist(str(row['q2_Title'])+ " " +str(row['q2_Body'])+ " " +str(row['q2_AcceptedAnswerBody'])+" " + str(row['Q1_image_text_new']), remove_stopwords=True, stem_words=False, remove_punc=True)
    q2_process.append(q2_Body)
    q2_image_text.append(text_to_wordlist(str(row['Q2_image_text_new']), remove_stopwords=True, stem_words=False, remove_punc=True))
    
    count+=1
    
print(count)

df["q1_process"] = q1_process
df["q2_process"] = q2_process
df["q1_image_text"] = q1_image_text
df["q2_image_text"] = q2_image_text

dict={'Id':list(df['Id']), 'q1_text_process':list(df['q1_process']), 'q2_text_process':list(df['q2_process']),
      'q1_image_text':list(df['q1_image_text']),'q2_image_text':list(df['q2_image_text']),
      'class':list(df['Class']) }
df=pd.DataFrame(dict)

In [149]:
import csv
import nltk
import numpy as np
from gensim.models.keyedvectors import KeyedVectors
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

In [150]:
print("loading Model . . . ")
model =   KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
print("google w2v loaded")

loading Model . . . 
google w2v loaded


In [151]:
from gensim.models import Word2Vec
from gensim import models
embeddings_index = {}
SO_file = 'textModel'
model2 = models.KeyedVectors.load_word2vec_format('SO_vectors_200.bin',binary=True)
print("SO w2v loaded")

SO w2v loaded


In [152]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 1), tokenizer=nltk.word_tokenize,
                                             analyzer="word", lowercase=True)
tfidf_vectorizer.fit_transform(list(df['q1_text_process']) + list(df['q2_text_process']))

its_vectorizer = TfidfVectorizer(ngram_range=(1, 1), tokenizer=nltk.word_tokenize,
                                             analyzer="word", lowercase=True) 
its_vectorizer.fit_transform(list(df['q1_image_text']) + list(df['q2_image_text']))
print("MODEL fitted!")

MODEL fitted!


In [153]:
import math
def soft_w2v(q1, q2, model,fittedTFIDF):
    if len(q1)==0 or len(q2)==0: return 0
    matrix0, matrix1 ,featureNames = vectorizer(q1 , q2, fittedTFIDF)
    index = []
    for i in np.nonzero(matrix0)[0]:
        index.append(i)
    for i in np.nonzero(matrix1)[0]:
        index.append(i)
    index = sorted(set(index))
    new_matrix0 = []
    new_matrix1 = []
    for l in index:
        new_matrix0.append(matrix0[l])
        new_matrix1.append(matrix1[l])
    docs = []
    docs.append(q1)
    docs.append(q2)
    featureNameLSZT = []
    for g in index:
        featureNameLSZT.append(featureNames[g])
    matrix = matrixByw2v(featureNameLSZT, model)
    npM0 = np.asarray(new_matrix0)
    npM1 = np.asarray(new_matrix1)
    tnpM0 = npM0.transpose()
    tnpM1 = npM1.transpose()
    dotProduct1 = np.dot(tnpM0, matrix)
    dotproduct2 = np.dot(dotProduct1, npM1)
    firstDomPartOne = np.dot(tnpM0, matrix)
    firstDomPartTwo = np.dot(firstDomPartOne, npM0)
    firstDominator = math.sqrt(firstDomPartTwo)
    secondDomPartOne = np.dot(tnpM1, matrix)
    secondDomPartTwo = np.dot(secondDomPartOne, npM1)
    secondDominator = math.sqrt(secondDomPartTwo)
    softCosineSimilarity = dotproduct2 / (firstDominator * secondDominator)
    if np.count_nonzero(new_matrix0)==0 or np.count_nonzero(new_matrix1)==0:
        return 0
    return softCosineSimilarity

def matrixByw2v(featureNames, model):

    matrix = np.zeros((len(featureNames), len(featureNames))).astype('float')
    # matrix =[[]]
    for i in range(len(featureNames)):
        for j in range(len(featureNames)):
            if i == j:
                matrix[i][j] = 1
            else:
                try:
                    sim = model.similarity(featureNames[i], featureNames[j])
                except KeyError:
                    sim = 0
                matrix[i][j] = (max(0, sim) * max(0, sim))
    return matrix

def vectorizer(question, parentQuestion, fittedTFIDF):
    if len(question) == 0 or len(parentQuestion) == 0:
        return [0]
    documents = []
    documents.append(question)
    documents.append(parentQuestion)
    tfidf_matrix = fittedTFIDF.transform(documents).toarray()
    featureNames = fittedTFIDF.get_feature_names()
    return tfidf_matrix[0], tfidf_matrix[1] ,featureNames

In [154]:
import Levenshtein
def soft_lev(q1, q2, model,fittedTFIDF):
    if len(q1)==0 or len(q2)==0: return 0
    matrix0, matrix1 ,featureNames = vectorizer(q1 , q2, fittedTFIDF)
    index = []
    for i in np.nonzero(matrix0)[0]:
        index.append(i)
    for i in np.nonzero(matrix1)[0]:
        index.append(i)
    index = sorted(set(index))
    new_matrix0 = []
    new_matrix1 = []
    for l in index:
        new_matrix0.append(matrix0[l])
        new_matrix1.append(matrix1[l])
    docs = []
    docs.append(q1)
    docs.append(q2)
    featureNameLSZT = []
    for g in index:
        featureNameLSZT.append(featureNames[g])
    matrix = matrixByLev(featureNameLSZT, model)
    npM0 = np.asarray(new_matrix0)
    npM1 = np.asarray(new_matrix1)
    tnpM0 = npM0.transpose()
    tnpM1 = npM1.transpose()
    dotProduct1 = np.dot(tnpM0, matrix)
    dotproduct2 = np.dot(dotProduct1, npM1)
    firstDomPartOne = np.dot(tnpM0, matrix)
    firstDomPartTwo = np.dot(firstDomPartOne, npM0)
    firstDominator = math.sqrt(firstDomPartTwo)
    secondDomPartOne = np.dot(tnpM1, matrix)
    secondDomPartTwo = np.dot(secondDomPartOne, npM1)
    secondDominator = math.sqrt(secondDomPartTwo)
    softCosineSimilarity = dotproduct2 / (firstDominator * secondDominator)
    if np.count_nonzero(new_matrix0)==0 or np.count_nonzero(new_matrix1)==0:
        return 0
    return softCosineSimilarity

def matrixByLev(featureNames, model):

    matrix = np.zeros((len(featureNames), len(featureNames))).astype('float')

    for i in range(len(featureNames)):
        for j in range(len(featureNames)):

            if i == j:
                matrix[i][j] = 1
            else:

                try:
                    lev = Levenshtein.distance(featureNames[i], featureNames[j])
                    lenfirst = len(featureNames[i])
                    lensecond = len(featureNames[j])
                    matrix[i][j] = 1.8 * (math.pow(1 - lev / max(lenfirst, lensecond), 5))
                except KeyError:
                    print("error in edit distance")


    return matrix

In [155]:
from sklearn.metrics.pairwise import cosine_similarity
def cos_similarity(q1, q2, model,fittedTFIDF):
    
    
    if len(q1)==0 or len(q2) ==0:
        return 0
    documents =[]
    documents.append(q1)
    documents.append(q2)
    tfidf_matrix = fittedTFIDF.transform(documents)
    matrix = cosine_similarity(tfidf_matrix[0,:], tfidf_matrix[1,:])
    return float(matrix[0][0])

In [262]:
soft_google_title =[]
soft_SO_title =[]
soft_leve_title = []
soft_cos_title = []
soft_google_body =[]
soft_SO_body =[]
soft_leve_body = []
soft_cos_body = []
soft_google_answer =[]
soft_SO_answer =[]
soft_leve_answer = []
soft_cos_answer = []

In [ ]:
count = 0
soft_google =[]
soft_SO =[]
soft_leve = []
soft_cos = []
soft_google_it =[]
soft_SO_it =[]
soft_leve_it = []
soft_cos_it = []

for i in range(len(df['q1_text_process'])):
    
        postText = df['q1_text_process'][i] 
        
        relatedPostText =  df['q2_text_process'][i]
        
        postText_image = df['q1_image_text'][i]
        
        relatedPostText_image = df['q2_image_text'][i]
        

        softw2v1 =soft_w2v(postText, relatedPostText, model, tfidf_vectorizer)

        soft_google.append(softw2v1)

        softw2v2 = soft_w2v(postText, relatedPostText, model2, tfidf_vectorizer)
        
        soft_SO.append(softw2v2)

        
        softlev = soft_lev(postText, relatedPostText, model2, tfidf_vectorizer)
        
        soft_leve.append(softlev)
        
        soft_cos.append(cos_similarity(postText, relatedPostText, model, tfidf_vectorizer))
        
        soft_google_it.append(soft_w2v(postText_image, relatedPostText_image, model, its_vectorizer))
        
        soft_SO_it.append(soft_w2v(postText_image, relatedPostText_image, model2, its_vectorizer))
        
        soft_leve_it.append(soft_lev(postText_image, relatedPostText_image, model2, its_vectorizer))
        
        soft_cos_it.append(cos_similarity(postText_image, relatedPostText_image, model, its_vectorizer))
    
        count = count + 1
        
        print (count)

In [157]:
dict1={'Id':list(df['Id']), 'class':list(df['class']), 'soft_google':soft_google,'soft_SO':soft_SO,'soft_leve':soft_leve,
      'soft_cos':soft_cos, 'soft_google_it':soft_google_it,'soft_SO_it':soft_SO_it,'soft_leve_it':soft_leve_it,
      'soft_cos_it':soft_cos_it}
model_data=pd.DataFrame(dict1)

In [192]:
df_2=pd.read_csv("RQ3_relatedness_Filtered_data.csv", encoding = 'latin-1', engine ='python')

dict_labels_source = {"Desktop App image":0, "Web browser image":1, "Mobile App image":2, "User-created Image":3}
dict_labels_purpose = {"desired output": 0, "context":1}
dict_labels_content = {"User Interface": 0, "Results":1}
q1_source=[]
q2_source=[]
q1_purpose=[]
q2_purpose=[]
q1_content=[]
q2_content=[]
for i in range(len(df_2['Id'])):
    q1_source.append(dict_labels_source[df_2['q1_source'][i]])
    q2_source.append(dict_labels_source[df_2['q2_source'][i]])
    q1_purpose.append(dict_labels_purpose[df_2['q1_purpose'][i]])
    q2_purpose.append(dict_labels_purpose[df_2['q2_purpose'][i]])
    q1_content.append(dict_labels_content[df_2['q1_content'][i]])
    q2_content.append(dict_labels_content[df_2['q2_content'][i]])
    

model_data['image_sim'] = list(df_2['Similarity'])
model_data['q1_source'] = list(q1_source)
model_data['q2_source'] = list(q2_source)
model_data['q1_purpose'] = list(q1_purpose)
model_data['q2_purpose'] = list(q2_purpose)
model_data['q1_content'] = list(q1_content)
model_data['q2_content'] = list(q2_content)

In [194]:
model_data

,Id,class,soft_google,soft_SO,soft_leve,soft_cos,soft_google_it,soft_SO_it,soft_leve_it,soft_cos_it,image_sim,q1_source,q2_source,q1_purpose,q2_purpose,q1_content,q2_content
0,274,duplicate,0.412504,0.589384,0.370031,0.281318,0.000000,0.000000,0.000000,0.000000,65,0,0,0,0,0,1
1,314,duplicate,0.813510,0.855175,0.728713,0.695153,0.243018,0.349820,0.151801,0.125719,82,0,0,0,0,1,1
2,584,duplicate,0.710846,0.827482,0.546367,0.427219,0.000000,0.000000,0.000000,0.000000,78,0,0,0,0,0,0
3,605,duplicate,0.888542,0.906706,0.869420,0.823088,0.782101,0.816453,0.795075,0.731419,93,0,0,1,1,1,0
4,606,duplicate,0.735209,0.728667,0.640454,0.521347,0.266725,0.288690,0.159834,0.041102,75,1,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1845,343612,isolated,0.394812,0.284807,0.117993,0.050746,0.079116,0.103170,0.014101,0.000000,66,0,2,0,0,1,0
1846,344643,isolated,0.686622,0.594806,0.555851,0.514593,0.245093,0.286436,0.064742,0.007375,67,0,1,1,1,0,0
1847,344649,isolated,0.663317,0.533914,0.488887,0.403396,0.057933,0.053867,0.011848,0.000000,72,0,0,1,0,0,0
1848,345398,isolated,0.558070,0.457995,0.226403,0.127004,0.067871,0.036150,0.023974,0.000000,84,1,0,0,0,1,1


<h1>TBA</h1>

In [261]:
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X = model_data[['soft_google','soft_SO','soft_leve','soft_cos']]
Y = model_data[['class']]

In [262]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2,random_state=880)

X_train, Y_train = oversample.fit_resample(X_train,Y_train)
from sklearn import svm

#Create a svm Classifier
clf = svm.SVC(kernel='linear')
clf.fit(X_train, Y_train)
y_pred = clf.predict(X_test)
from sklearn import metrics
print (metrics.accuracy_score(Y_test, y_pred))
print(metrics.classification_report(Y_test, y_pred))

0.34594594594594597
              precision    recall  f1-score   support

      direct       0.68      0.22      0.33       186
   duplicate       0.16      0.64      0.25        42
    indirect       0.40      0.35      0.37       101
    isolated       0.51      0.61      0.56        41

    accuracy                           0.35       370
   macro avg       0.44      0.45      0.38       370
weighted avg       0.53      0.35      0.36       370



<h1>TBA + IS</h1>

In [216]:
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X = model_data[['soft_google','soft_SO','soft_leve','soft_cos','image_sim']]
Y = model_data[['class']]

In [217]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2,random_state=880)

X_train, Y_train = oversample.fit_resample(X_train,Y_train)
from sklearn import svm

#Create a svm Classifier
clf = svm.SVC(kernel='linear')
clf.fit(X_train, Y_train)
y_pred = clf.predict(X_test)
from sklearn import metrics
print (metrics.accuracy_score(Y_test, y_pred))
print(metrics.classification_report(Y_test, y_pred))

0.34054054054054056
              precision    recall  f1-score   support

      direct       0.62      0.20      0.30       186
   duplicate       0.18      0.69      0.28        42
    indirect       0.35      0.33      0.34       101
    isolated       0.50      0.66      0.57        41

    accuracy                           0.34       370
   macro avg       0.41      0.47      0.37       370
weighted avg       0.48      0.34      0.34       370



<h1>TBA+IS+ITS</h1>

In [218]:
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X = model_data[['soft_google','soft_SO','soft_leve','soft_cos','image_sim', 
                'soft_google_it','soft_SO_it','soft_leve_it','soft_cos_it']]
Y = model_data[['class']]

In [219]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2,random_state=880)

X_train, Y_train = oversample.fit_resample(X_train,Y_train)
from sklearn import svm

#Create a svm Classifier
clf = svm.SVC(kernel='linear')
clf.fit(X_train, Y_train)
y_pred = clf.predict(X_test)
from sklearn import metrics
print (metrics.accuracy_score(Y_test, y_pred))
print(metrics.classification_report(Y_test, y_pred))

0.44594594594594594
              precision    recall  f1-score   support

      direct       0.77      0.40      0.52       186
   duplicate       0.18      0.48      0.26        42
    indirect       0.41      0.44      0.42       101
    isolated       0.47      0.66      0.55        41

    accuracy                           0.45       370
   macro avg       0.46      0.49      0.44       370
weighted avg       0.57      0.45      0.47       370



<h1>TBA+S+C+P</h1>

In [220]:
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X = model_data[['soft_google','soft_SO','soft_leve','soft_cos',
                'q1_source','q2_source','q1_purpose','q2_purpose','q1_content','q2_content']]
Y = model_data[['class']]

In [221]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2,random_state=880)

X_train, Y_train = oversample.fit_resample(X_train,Y_train)
from sklearn import svm

#Create a svm Classifier
clf = svm.SVC(kernel='linear')
clf.fit(X_train, Y_train)
y_pred = clf.predict(X_test)
from sklearn import metrics
print (metrics.accuracy_score(Y_test, y_pred))
print(metrics.classification_report(Y_test, y_pred))

0.4243243243243243
              precision    recall  f1-score   support

      direct       0.61      0.40      0.48       186
   duplicate       0.23      0.55      0.33        42
    indirect       0.38      0.35      0.36       101
    isolated       0.43      0.61      0.51        41

    accuracy                           0.42       370
   macro avg       0.41      0.48      0.42       370
weighted avg       0.49      0.42      0.43       370



<h1>TBA+IS+S+C+P</h1>

In [223]:
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X = model_data[['soft_google','soft_SO','soft_leve','soft_cos', 'image_sim',
                'q1_source','q2_source','q1_purpose','q2_purpose','q1_content','q2_content']]
Y = model_data[['class']]

In [224]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2,random_state=880)

X_train, Y_train = oversample.fit_resample(X_train,Y_train)
from sklearn import svm

#Create a svm Classifier
clf = svm.SVC(kernel='linear')
clf.fit(X_train, Y_train)
y_pred = clf.predict(X_test)
from sklearn import metrics
print (metrics.accuracy_score(Y_test, y_pred))
print(metrics.classification_report(Y_test, y_pred))

0.4486486486486487
              precision    recall  f1-score   support

      direct       0.64      0.39      0.49       186
   duplicate       0.24      0.50      0.33        42
    indirect       0.40      0.47      0.43       101
    isolated       0.48      0.61      0.54        41

    accuracy                           0.45       370
   macro avg       0.44      0.49      0.45       370
weighted avg       0.51      0.45      0.46       370



<h1>TBA+ITS+S+C+P</h1>

In [225]:
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X = model_data[['soft_google','soft_SO','soft_leve','soft_cos',
                'soft_google_it','soft_SO_it','soft_leve_it','soft_cos_it',
                'q1_source','q2_source','q1_purpose','q2_purpose','q1_content','q2_content']]
Y = model_data[['class']]

In [226]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2,random_state=880)

X_train, Y_train = oversample.fit_resample(X_train,Y_train)
from sklearn import svm

#Create a svm Classifier
clf = svm.SVC(kernel='linear')
clf.fit(X_train, Y_train)
y_pred = clf.predict(X_test)
from sklearn import metrics
print (metrics.accuracy_score(Y_test, y_pred))
print(metrics.classification_report(Y_test, y_pred))

0.4540540540540541
              precision    recall  f1-score   support

      direct       0.69      0.42      0.52       186
   duplicate       0.21      0.55      0.31        42
    indirect       0.43      0.38      0.40       101
    isolated       0.48      0.68      0.57        41

    accuracy                           0.45       370
   macro avg       0.45      0.51      0.45       370
weighted avg       0.54      0.45      0.47       370



<h1>TBA+ITS+S</h1>

In [227]:
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X = model_data[['soft_google','soft_SO','soft_leve','soft_cos',
                'soft_google_it','soft_SO_it','soft_leve_it','soft_cos_it',
                'q1_source','q2_source']]
Y = model_data[['class']]

In [228]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2,random_state=880)

X_train, Y_train = oversample.fit_resample(X_train,Y_train)
from sklearn import svm

#Create a svm Classifier
clf = svm.SVC(kernel='linear')
clf.fit(X_train, Y_train)
y_pred = clf.predict(X_test)
from sklearn import metrics
print (metrics.accuracy_score(Y_test, y_pred))
print(metrics.classification_report(Y_test, y_pred))

0.4027027027027027
              precision    recall  f1-score   support

      direct       0.71      0.35      0.47       186
   duplicate       0.18      0.52      0.26        42
    indirect       0.38      0.35      0.36       101
    isolated       0.43      0.63      0.51        41

    accuracy                           0.40       370
   macro avg       0.42      0.46      0.40       370
weighted avg       0.53      0.40      0.42       370



<h1>TBA+ITS+C</h1>

In [229]:
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X = model_data[['soft_google','soft_SO','soft_leve','soft_cos',
                'soft_google_it','soft_SO_it','soft_leve_it','soft_cos_it',
                'q1_content','q2_content']]
Y = model_data[['class']]

In [230]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2,random_state=880)

X_train, Y_train = oversample.fit_resample(X_train,Y_train)
from sklearn import svm

#Create a svm Classifier
clf = svm.SVC(kernel='linear')
clf.fit(X_train, Y_train)
y_pred = clf.predict(X_test)
from sklearn import metrics
print (metrics.accuracy_score(Y_test, y_pred))
print(metrics.classification_report(Y_test, y_pred))

0.45135135135135135
              precision    recall  f1-score   support

      direct       0.76      0.37      0.49       186
   duplicate       0.19      0.52      0.28        42
    indirect       0.44      0.49      0.46       101
    isolated       0.52      0.68      0.59        41

    accuracy                           0.45       370
   macro avg       0.48      0.51      0.46       370
weighted avg       0.58      0.45      0.47       370



<h1>TBA+ITS+P</h1>

In [233]:
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X = model_data[['soft_google','soft_SO','soft_leve','soft_cos',
                'soft_google_it','soft_SO_it','soft_leve_it','soft_cos_it',
                'q1_purpose','q2_purpose']]
Y = model_data[['class']]

In [234]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2,random_state=880)

X_train, Y_train = oversample.fit_resample(X_train,Y_train)
from sklearn import svm

#Create a svm Classifier
clf = svm.SVC(kernel='linear')
clf.fit(X_train, Y_train)
y_pred = clf.predict(X_test)
from sklearn import metrics
print (metrics.accuracy_score(Y_test, y_pred))
print(metrics.classification_report(Y_test, y_pred))

0.4702702702702703
              precision    recall  f1-score   support

      direct       0.69      0.51      0.58       186
   duplicate       0.19      0.36      0.25        42
    indirect       0.40      0.38      0.39       101
    isolated       0.46      0.66      0.54        41

    accuracy                           0.47       370
   macro avg       0.43      0.47      0.44       370
weighted avg       0.53      0.47      0.49       370



<h1>TBA+ITS+S+C</h1>

In [235]:
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X = model_data[['soft_google','soft_SO','soft_leve','soft_cos',
                'soft_google_it','soft_SO_it','soft_leve_it','soft_cos_it',
                'q1_source','q2_source','q1_content','q2_content']]
Y = model_data[['class']]

In [236]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2,random_state=880)

X_train, Y_train = oversample.fit_resample(X_train,Y_train)
from sklearn import svm

#Create a svm Classifier
clf = svm.SVC(kernel='linear')
clf.fit(X_train, Y_train)
y_pred = clf.predict(X_test)
from sklearn import metrics
print (metrics.accuracy_score(Y_test, y_pred))
print(metrics.classification_report(Y_test, y_pred))

0.4081081081081081
              precision    recall  f1-score   support

      direct       0.70      0.32      0.44       186
   duplicate       0.20      0.57      0.30        42
    indirect       0.39      0.40      0.39       101
    isolated       0.44      0.66      0.52        41

    accuracy                           0.41       370
   macro avg       0.43      0.49      0.41       370
weighted avg       0.53      0.41      0.42       370



<h1>TBA+ITS+S+P</h1>

In [237]:
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X = model_data[['soft_google','soft_SO','soft_leve','soft_cos',
                'soft_google_it','soft_SO_it','soft_leve_it','soft_cos_it',
                'q1_source','q2_source','q1_purpose','q2_purpose']]
Y = model_data[['class']]

In [238]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2,random_state=880)

X_train, Y_train = oversample.fit_resample(X_train,Y_train)
from sklearn import svm

#Create a svm Classifier
clf = svm.SVC(kernel='linear')
clf.fit(X_train, Y_train)
y_pred = clf.predict(X_test)
from sklearn import metrics
print (metrics.accuracy_score(Y_test, y_pred))
print(metrics.classification_report(Y_test, y_pred))

0.44054054054054054
              precision    recall  f1-score   support

      direct       0.73      0.37      0.49       186
   duplicate       0.19      0.52      0.28        42
    indirect       0.44      0.45      0.44       101
    isolated       0.44      0.66      0.53        41

    accuracy                           0.44       370
   macro avg       0.45      0.50      0.44       370
weighted avg       0.56      0.44      0.46       370



<h1>TBA+ITS+C+P</h1>

In [242]:
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X = model_data[['soft_google','soft_SO','soft_leve','soft_cos',
                'soft_google_it','soft_SO_it','soft_leve_it','soft_cos_it',
                'q1_purpose','q2_purpose','q1_content','q2_content']]
Y = model_data[['class']]

In [243]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2,random_state=880)

X_train, Y_train = oversample.fit_resample(X_train,Y_train)
from sklearn import svm

#Create a svm Classifier
clf = svm.SVC(kernel='linear')
clf.fit(X_train, Y_train)
y_pred = clf.predict(X_test)
from sklearn import metrics
print (metrics.accuracy_score(Y_test, y_pred))
print(metrics.classification_report(Y_test, y_pred))

0.46756756756756757
              precision    recall  f1-score   support

      direct       0.65      0.49      0.56       186
   duplicate       0.22      0.48      0.30        42
    indirect       0.41      0.34      0.37       101
    isolated       0.50      0.66      0.57        41

    accuracy                           0.47       370
   macro avg       0.45      0.49      0.45       370
weighted avg       0.52      0.47      0.48       370



<h1>TBA+IS+ITS+S+C+P</h1>

In [246]:
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X = model_data[['soft_google','soft_SO','soft_leve','soft_cos','image_sim',
                'soft_google_it','soft_SO_it','soft_leve_it','soft_cos_it',
                'q1_source','q2_source','q1_purpose','q2_purpose','q1_content','q2_content']]
Y = model_data[['class']]

In [247]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2,random_state=880)

X_train, Y_train = oversample.fit_resample(X_train,Y_train)
from sklearn import svm

#Create a svm Classifier
clf = svm.SVC(kernel='linear')
clf.fit(X_train, Y_train)
y_pred = clf.predict(X_test)
from sklearn import metrics
print (metrics.accuracy_score(Y_test, y_pred))
print(metrics.classification_report(Y_test, y_pred))

0.43243243243243246
              precision    recall  f1-score   support

      direct       0.65      0.37      0.47       186
   duplicate       0.23      0.55      0.32        42
    indirect       0.39      0.44      0.41       101
    isolated       0.50      0.61      0.55        41

    accuracy                           0.43       370
   macro avg       0.44      0.49      0.44       370
weighted avg       0.51      0.43      0.44       370



<h1>TBA+IS+ITS+S</h1>

In [248]:
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X = model_data[['soft_google','soft_SO','soft_leve','soft_cos','image_sim',
                'soft_google_it','soft_SO_it','soft_leve_it','soft_cos_it',
                'q1_source','q2_source']]
Y = model_data[['class']]

In [249]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2,random_state=880)

X_train, Y_train = oversample.fit_resample(X_train,Y_train)
from sklearn import svm

#Create a svm Classifier
clf = svm.SVC(kernel='linear')
clf.fit(X_train, Y_train)
y_pred = clf.predict(X_test)
from sklearn import metrics
print (metrics.accuracy_score(Y_test, y_pred))
print(metrics.classification_report(Y_test, y_pred))

0.3675675675675676
              precision    recall  f1-score   support

      direct       0.61      0.28      0.38       186
   duplicate       0.17      0.60      0.26        42
    indirect       0.40      0.34      0.36       101
    isolated       0.49      0.61      0.54        41

    accuracy                           0.37       370
   macro avg       0.42      0.46      0.39       370
weighted avg       0.49      0.37      0.38       370



<h1>TBA+IS+ITS+C</h1>

In [250]:
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X = model_data[['soft_google','soft_SO','soft_leve','soft_cos','image_sim',
                'soft_google_it','soft_SO_it','soft_leve_it','soft_cos_it',
                'q1_content','q2_content']]
Y = model_data[['class']]

In [251]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2,random_state=880)

X_train, Y_train = oversample.fit_resample(X_train,Y_train)
from sklearn import svm

#Create a svm Classifier
clf = svm.SVC(kernel='linear')
clf.fit(X_train, Y_train)
y_pred = clf.predict(X_test)
from sklearn import metrics
print (metrics.accuracy_score(Y_test, y_pred))
print(metrics.classification_report(Y_test, y_pred))

0.4756756756756757
              precision    recall  f1-score   support

      direct       0.72      0.45      0.56       186
   duplicate       0.21      0.45      0.29        42
    indirect       0.41      0.46      0.43       101
    isolated       0.51      0.66      0.57        41

    accuracy                           0.48       370
   macro avg       0.46      0.50      0.46       370
weighted avg       0.56      0.48      0.49       370



<h1>TBA+IS+ITS+P</h1>

In [252]:
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X = model_data[['soft_google','soft_SO','soft_leve','soft_cos','image_sim',
                'soft_google_it','soft_SO_it','soft_leve_it','soft_cos_it',
                'q1_purpose','q2_purpose']]
Y = model_data[['class']]

In [253]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2,random_state=880)

X_train, Y_train = oversample.fit_resample(X_train,Y_train)
from sklearn import svm

#Create a svm Classifier
clf = svm.SVC(kernel='linear')
clf.fit(X_train, Y_train)
y_pred = clf.predict(X_test)
from sklearn import metrics
print (metrics.accuracy_score(Y_test, y_pred))
print(metrics.classification_report(Y_test, y_pred))

0.4891891891891892
              precision    recall  f1-score   support

      direct       0.74      0.46      0.57       186
   duplicate       0.22      0.43      0.29        42
    indirect       0.43      0.48      0.45       101
    isolated       0.48      0.71      0.57        41

    accuracy                           0.49       370
   macro avg       0.47      0.52      0.47       370
weighted avg       0.57      0.49      0.51       370



<h1>TBA+IS+ITS+S+C</h1>

In [254]:
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X = model_data[['soft_google','soft_SO','soft_leve','soft_cos','image_sim',
                'soft_google_it','soft_SO_it','soft_leve_it','soft_cos_it',
                'q1_source','q2_source','q1_content','q2_content']]
Y = model_data[['class']]

In [255]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2,random_state=880)

X_train, Y_train = oversample.fit_resample(X_train,Y_train)
from sklearn import svm

#Create a svm Classifier
clf = svm.SVC(kernel='linear')
clf.fit(X_train, Y_train)
y_pred = clf.predict(X_test)
from sklearn import metrics
print (metrics.accuracy_score(Y_test, y_pred))
print(metrics.classification_report(Y_test, y_pred))

0.3972972972972973
              precision    recall  f1-score   support

      direct       0.62      0.29      0.40       186
   duplicate       0.20      0.57      0.29        42
    indirect       0.39      0.44      0.41       101
    isolated       0.50      0.61      0.55        41

    accuracy                           0.40       370
   macro avg       0.43      0.48      0.41       370
weighted avg       0.50      0.40      0.41       370



<h1>TBA+IS+ITS+S+P</h1>

In [256]:
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X = model_data[['soft_google','soft_SO','soft_leve','soft_cos','image_sim',
                'soft_google_it','soft_SO_it','soft_leve_it','soft_cos_it',
                'q1_source','q2_source','q1_purpose','q2_purpose']]
Y = model_data[['class']]

In [257]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2,random_state=880)

X_train, Y_train = oversample.fit_resample(X_train,Y_train)
from sklearn import svm

#Create a svm Classifier
clf = svm.SVC(kernel='linear')
clf.fit(X_train, Y_train)
y_pred = clf.predict(X_test)
from sklearn import metrics
print (metrics.accuracy_score(Y_test, y_pred))
print(metrics.classification_report(Y_test, y_pred))

0.4135135135135135
              precision    recall  f1-score   support

      direct       0.66      0.34      0.45       186
   duplicate       0.19      0.55      0.28        42
    indirect       0.41      0.41      0.41       101
    isolated       0.49      0.61      0.54        41

    accuracy                           0.41       370
   macro avg       0.44      0.48      0.42       370
weighted avg       0.52      0.41      0.43       370



<h1>TBA+IS+ITS+C+P</h1>

In [259]:
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X = model_data[['soft_google','soft_SO','soft_leve','soft_cos','image_sim',
                'soft_google_it','soft_SO_it','soft_leve_it','soft_cos_it',
                'q1_purpose','q2_purpose','q1_content','q2_content']]
Y = model_data[['class']]

In [260]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2,random_state=880)

X_train, Y_train = oversample.fit_resample(X_train,Y_train)
from sklearn import svm

#Create a svm Classifier
clf = svm.SVC(kernel='linear')
clf.fit(X_train, Y_train)
y_pred = clf.predict(X_test)
from sklearn import metrics
print (metrics.accuracy_score(Y_test, y_pred))
print(metrics.classification_report(Y_test, y_pred))

0.4837837837837838
              precision    recall  f1-score   support

      direct       0.79      0.40      0.53       186
   duplicate       0.25      0.50      0.33        42
    indirect       0.41      0.56      0.47       101
    isolated       0.51      0.66      0.57        41

    accuracy                           0.48       370
   macro avg       0.49      0.53      0.48       370
weighted avg       0.59      0.48      0.50       370

